In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Acre - Produção de Cimento (t)'], axis=1)
data

,Ano,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),...,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Renda,Acre - Consumo de Cimento (t)
0,2003,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,5.122550e+09,7.083133e+09,...,0.691320,0.773065,0.643805,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.651726,43.603000
1,2004,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,5.449927e+09,7.384787e+09,...,0.694888,0.777735,0.649379,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06,0.653176,58.142000
2,2005,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,6.065174e+09,7.948531e+09,...,0.698098,0.781889,0.654591,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06,0.654500,80.914000
3,2006,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,6.842013e+09,8.653759e+09,...,0.700957,0.785537,0.659437,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06,0.655703,114.415000
4,2007,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,7.654167e+09,9.379862e+09,...,0.703473,0.788689,0.663911,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06,0.656789,121.872000
5,2008,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,8.375355e+09,1.000623e+10,...,0.705654,0.791356,0.668008,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.657765,119.145000
6,2009,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,8.879299e+09,1.041227e+10,...,0.707507,0.793548,0.671723,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.658635,121.146000
7,2010,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,9.039721e+09,1.047735e+10,...,0.709040,0.795275,0.675049,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.659404,174.705000
8,2011,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,8.730341e+09,1.008089e+10,...,0.710262,0.796547,0.677982,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.660077,178.322000
9,2012,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,7.824880e+09,9.102259e+09,...,0.711004,0.797243,0.680191,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.660660,173.494000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,INCC (%),Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Renda
0,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,5.122550e+09,7.083133e+09,1.562192e+12,...,0.895303,0.691320,0.773065,0.643805,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.651726
1,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,5.449927e+09,7.384787e+09,1.574016e+12,...,0.746570,0.694888,0.777735,0.649379,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06,0.653176
2,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,6.065174e+09,7.948531e+09,1.606744e+12,...,0.589811,0.698098,0.781889,0.654591,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06,0.654500
3,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,6.842013e+09,8.653759e+09,1.668478e+12,...,0.606044,0.700957,0.785537,0.659437,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06,0.655703
4,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,7.654167e+09,9.379862e+09,1.767318e+12,...,0.633401,0.703473,0.788689,0.663911,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06,0.656789
5,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,8.375355e+09,1.000623e+10,1.911366e+12,...,0.587619,0.705654,0.791356,0.668008,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.657765
6,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,8.879299e+09,1.041227e+10,2.108725e+12,...,0.566759,0.707507,0.793548,0.671723,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.658635
7,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,9.039721e+09,1.047735e+10,2.367495e+12,...,0.593537,0.709040,0.795275,0.675049,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.659404
8,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,8.730341e+09,1.008089e+10,2.695778e+12,...,0.600821,0.710262,0.796547,0.677982,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.660077
9,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,7.824880e+09,9.102259e+09,3.101676e+12,...,0.604731,0.711004,0.797243,0.680191,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.660660


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     173.494000
10    193.672000
11    171.709286
12    151.486629
13    134.223086
14    150.491000
15    148.696000
16    153.806000
17    178.033000
18    189.357000
19    202.321000
20           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,INCC (%),Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Renda
0,1.862218,-1.377227,-0.455887,-1.283032,-1.143593,-0.539011,-0.196467,-0.267673,-0.223571,-1.058141,...,2.660855,-2.067476,-2.186779,-1.917077,-0.791179,-1.702029,0.462834,-0.475474,-2.146899,-2.121614
1,1.610564,-1.169882,-0.330414,-1.244132,-0.955546,-0.374598,-0.271137,-0.142319,-0.093175,-1.049773,...,1.389772,-1.630886,-1.688539,-1.573231,-0.828061,-1.465408,0.668944,-0.168409,-1.675488,-1.698110
2,1.166911,-0.785870,-0.091853,-1.181300,-0.602298,-0.065950,-0.410307,0.093262,0.150516,-1.026610,...,0.050101,-1.238167,-1.245379,-1.251645,-0.848437,-1.242634,0.616017,0.200353,-1.250308,-1.311447
3,0.612524,-0.292998,0.214312,-1.090696,-0.148064,0.322729,-0.584021,0.390718,0.455366,-0.982919,...,0.188826,-0.888369,-0.856201,-0.952657,-0.898403,-1.039661,0.860434,0.611375,-0.907826,-0.960151
4,0.028666,0.240927,0.542596,-0.968483,0.342942,0.727234,-0.755089,0.701695,0.769241,-0.912965,...,0.422624,-0.580542,-0.519907,-0.676604,-0.904884,-0.847483,1.036581,1.104733,-0.565413,-0.642746
5,-0.503399,0.748099,0.847513,-0.810821,0.806504,1.083359,-0.797208,0.977842,1.040004,-0.811016,...,0.031364,-0.313737,-0.235401,-0.423821,-0.922380,-0.693086,1.230946,1.251237,-0.418480,-0.357754
6,-0.902408,1.160710,1.083581,-0.613871,1.178409,1.326899,-0.797208,1.170804,1.215520,-0.671338,...,-0.146903,-0.087004,-0.001584,-0.194646,-0.904912,-0.585505,1.393841,0.998864,-0.446981,-0.103701
7,-1.087096,1.410953,1.205313,-0.373795,1.394441,1.393649,-0.797208,1.232231,1.243656,-0.488195,...,0.081938,0.100605,0.182642,0.010583,-0.922610,-0.184381,0.758665,0.760393,0.059326,0.120889
8,-0.976201,1.431021,1.167226,-0.086755,1.390385,1.219403,-0.797208,1.113767,1.072274,-0.255856,...,0.144188,0.250041,0.318373,0.191532,-0.931118,0.186244,0.254274,0.638733,0.477015,0.317493
9,-0.488458,1.153106,0.923834,0.251090,1.102027,0.739957,-0.672757,0.767062,0.649240,0.031414,...,0.177601,0.340874,0.392660,0.327789,0.224200,0.533172,-0.061495,0.582031,0.883643,0.487586


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     173.494000
10    193.672000
11    171.709286
12    151.486629
13    134.223086
14    150.491000
15    148.696000
16    153.806000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,INCC (%),Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Renda
17,-0.718272,1.334994,-1.300027,1.151763,1.324067,-0.920678,1.78769,-0.935888,-0.74274,2.149335,...,0.050004,1.257012,1.018283,1.386931,1.209891,0.984767,-1.180137,-1.702195,0.425528,0.797596


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    178.033
Name: Acre - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(1, split_factor + 1):
        pos = df.shape[0] - (i*2 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input.iloc[:-1], 
                        train_target.iloc[:-1], 
                        epochs=10000,
                        validation_data=(train_input.iloc[-1:], 
                                         train_target.iloc[-1:]),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [65]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [ ]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3286875848, 3563561496, 2145643222, 1508199131, 4177795604, 2334309410, 2149495660, 1139825845, 1915134000, 4137867200, 3330260489, 941484801, 1774540029, 1330231576, 1285981757, 3506740544, 1774707720, 2229665694, 592658678, 544908178, 4095335498, 3764091786, 689396292, 4246006239, 3363573528, 1348827845, 2391925613, 2515208820, 731949471, 4294452280, 310092118, 3288640117, 1720069775, 1933827299, 2696703721, 1222588484, 4182373667, 2265704762, 624506082, 557606262, 94126282, 1841677970, 4158291386, 3051064206, 294788418, 369126759, 3567748066, 4202028225, 1766831652, 3070473509, 3527882659, 3196755439, 1247132923, 836177434, 1441562709, 1875040261, 129782309, 1899024138, 73726691, 2406968762, 1481790349, 853249807, 1822403974, 1006601218, 2436339702, 33630540, 3388642372, 2459163399, 1067517926, 4219139656, 1626493170, 2628839736, 1694313558, 39020962, 304900314, 3440093323, 2734708190, 1185438390, 4189517098, 3776478979, 4245098826, 2422940570, 1886403098, 3224040532, 4228842970, 1

In [ ]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

In [ ]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

In [ ]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [ ]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

In [ ]:
display(targets)
display(predictions)

In [ ]:
mae = mean_absolute_error(predictions, targets)
mae

In [ ]:
porcentage = mae/np.mean(targets)
porcentage